# TRAIN HANGUL-RNN

In [1]:
# -*- coding: utf-8 -*-
# Import Packages
import numpy as np
import tensorflow as tf
import collections
import argparse
import time
import os
import hgtk

from six.moves import cPickle
from TextLoader import *

print ("Packages Imported")

Packages Imported


# LOAD DATASET WITH TEXTLOADER

In [2]:
data_dir    = "data/nine_dreams"
batch_size  = 50
seq_length  = 50
data_loader = TextLoader(data_dir, batch_size, seq_length)
# This makes "vocab.pkl" and "data.npy" in "data/nine_dreams"   
#  from "data/nine_dreams/input.txt" 

loading preprocessed files


# VOCAB AND CHARS

In [3]:
vocab_size = data_loader.vocab_size
vocab = data_loader.vocab
chars = data_loader.chars
print ( "type of 'data_loader.vocab' is %s, length is %d" 
       % (type(data_loader.vocab), len(data_loader.vocab)) )
print ( "type of 'data_loader.chars' is %s, length is %d" 
       % (type(data_loader.chars), len(data_loader.chars)) )

type of 'data_loader.vocab' is <class 'dict'>, length is 76
type of 'data_loader.chars' is <class 'tuple'>, length is 76


# VOCAB: DICTIONARY (CHAR->INDEX)

In [4]:
print (data_loader.vocab)

{' ': 1, 'ㅇ': 2, 'ㄱ': 7, 'ㅔ': 20, 'ㅋ': 42, 'ㄲ': 29, 'ㅢ': 25, '-': 65, 'ㄹ': 5, '\x1a': 72, 'ㅈ': 15, 'ㅖ': 39, 'ㅐ': 21, '?': 41, ':': 57, '3': 66, '.': 24, 'ㅅ': 10, 'ㅍ': 31, '1': 58, 'ㅚ': 33, 'ㅄ': 43, '7': 71, 'ㄴ': 4, '5': 63, 'ㄿ': 56, '2': 62, 'ᴥ': 0, 'ㅏ': 3, 'ㅑ': 34, 'ㅀ': 51, 'ㅟ': 35, '6': 59, 'ㅆ': 22, 'ㄳ': 60, 'ㅂ': 17, 'ㅕ': 18, '(': 45, 'ㅞ': 61, 'ㅓ': 11, 'ㅡ': 8, '"': 28, 'ㅊ': 23, '!': 52, 'ㅒ': 74, '>': 75, 'ㅗ': 9, 'ㅃ': 47, 'ㅘ': 26, 'ㅝ': 36, '0': 73, '\n': 19, '4': 67, 'ㅌ': 30, 'ㅙ': 53, 'ㄻ': 55, 'ㄵ': 50, "'": 49, ',': 27, 'ㅠ': 37, '9': 64, 'ㅎ': 14, '_': 69, 'ㄶ': 44, 'ㅣ': 6, 'ㅁ': 12, 'ㄺ': 48, 'ㅜ': 16, 'ㅛ': 38, '8': 68, 'ㄾ': 70, ')': 46, 'ㅉ': 40, 'ㄸ': 32, 'ㄷ': 13, 'ㄼ': 54}


# CHARS: LIST (INDEX->CHAR)

In [5]:
print (data_loader.chars)
# USAGE
print (data_loader.chars[0])

('ᴥ', ' ', 'ㅇ', 'ㅏ', 'ㄴ', 'ㄹ', 'ㅣ', 'ㄱ', 'ㅡ', 'ㅗ', 'ㅅ', 'ㅓ', 'ㅁ', 'ㄷ', 'ㅎ', 'ㅈ', 'ㅜ', 'ㅂ', 'ㅕ', '\n', 'ㅔ', 'ㅐ', 'ㅆ', 'ㅊ', '.', 'ㅢ', 'ㅘ', ',', '"', 'ㄲ', 'ㅌ', 'ㅍ', 'ㄸ', 'ㅚ', 'ㅑ', 'ㅟ', 'ㅝ', 'ㅠ', 'ㅛ', 'ㅖ', 'ㅉ', '?', 'ㅋ', 'ㅄ', 'ㄶ', '(', ')', 'ㅃ', 'ㄺ', "'", 'ㄵ', 'ㅀ', '!', 'ㅙ', 'ㄼ', 'ㄻ', 'ㄿ', ':', '1', '6', 'ㄳ', 'ㅞ', '2', '5', '9', '-', '3', '4', '8', '_', 'ㄾ', '7', '\x1a', '0', 'ㅒ', '>')
ᴥ


# TRAINING BATCH (IMPORTANT!!)

In [6]:
x, y = data_loader.next_batch()
print ("Type of 'x' is %s. Shape is %s" % (type(x), x.shape,))
print ("x looks like \n%s" % (x))
print
print ("Type of 'y' is %s. Shape is %s" % (type(y), y.shape,))
print ("y looks like \n%s" % (y))

Type of 'x' is <class 'numpy.ndarray'>. Shape is (50, 50)
x looks like 
[[19 19  1 ..., 23  0  4]
 [ 1 19 42 ...,  6  0  2]
 [ 2  8  0 ..., 19  1  1]
 ..., 
 [ 1  7  8 ...,  2  6  0]
 [ 6  4  0 ..., 17  9  7]
 [ 1 19 17 ..., 17 16  5]]
Type of 'y' is <class 'numpy.ndarray'>. Shape is (50, 50)
y looks like 
[[19  1  1 ...,  0  4  3]
 [19 42 11 ...,  0  2 20]
 [ 8  0  5 ...,  1  1 10]
 ..., 
 [ 7  8  0 ...,  6  0  1]
 [ 4  0 12 ...,  9  7  0]
 [19 17 16 ..., 16  5  0]]


# DEFINE A MULTILAYER LSTM NETWORK

In [7]:
rnn_size   = 512
num_layers = 3
grad_clip  = 5. # <= GRADIENT CLIPPING (PRACTICALLY IMPORTANT)
vocab_size = data_loader.vocab_size

tf.reset_default_graph()

# SELECT RNN CELL (MULTI LAYER LSTM)
unitcell = tf.contrib.rnn.BasicLSTMCell(rnn_size)
cell = tf.contrib.rnn.MultiRNNCell([unitcell] * num_layers)

# Set paths to the graph
input_data = tf.placeholder(tf.int32, [batch_size, seq_length])
targets    = tf.placeholder(tf.int32, [batch_size, seq_length])
initial_state = cell.zero_state(batch_size, tf.float32)

# Set Network
with tf.variable_scope('rnnlm'):
    softmax_w = tf.get_variable("softmax_w", [rnn_size, vocab_size])
    softmax_b = tf.get_variable("softmax_b", [vocab_size])
    with tf.device("/cpu:0"):
        embedding = tf.get_variable("embedding", [vocab_size, rnn_size])
        inputs = tf.split(tf.nn.embedding_lookup(
                embedding, input_data), seq_length, 1)
        inputs = [tf.squeeze(input_, [1]) for input_ in inputs]
        
print ("Network ready")

Network ready


# Define functions

In [8]:
# Output of RNN
outputs, last_state = tf.contrib.legacy_seq2seq.rnn_decoder(inputs, initial_state
                        , cell, loop_function=None, scope='rnnlm')
output = tf.reshape(tf.concat(outputs, 1), [-1, rnn_size])
logits = tf.nn.xw_plus_b(output, softmax_w, softmax_b)

# Next word probability
probs = tf.nn.softmax(logits)
print ("FUNCTIONS READY")

FUNCTIONS READY


# DEFINE LOSS FUNCTION 

In [9]:
loss = tf.contrib.legacy_seq2seq.sequence_loss_by_example([logits], # Input
    [tf.reshape(targets, [-1])], # Target
    [tf.ones([batch_size * seq_length])], # Weight
    vocab_size)
print ("LOSS FUNCTION")

LOSS FUNCTION


# DEFINE COST FUNCTION 

In [ ]:
cost = tf.reduce_sum(loss) / batch_size / seq_length

# GRADIENT CLIPPING ! 
lr = tf.Variable(0.0, trainable=False) # <= LEARNING RATE 
tvars = tf.trainable_variables()
grads, _ = tf.clip_by_global_norm(tf.gradients(cost, tvars), grad_clip)
_optm = tf.train.AdamOptimizer(lr)
optm = _optm.apply_gradients(zip(grads, tvars))

final_state = last_state
print ("NETWORK READY")

NETWORK READY


# OPTIMIZE NETWORK WITH LR SCHEDULING

In [ ]:
num_epochs    = 500
save_every    = 1000
learning_rate = 0.0002
decay_rate    = 0.97

save_dir = 'data/nine_dreams'
sess = tf.Session()
sess.run(tf.global_variables_initializer())
summary_writer = tf.summary.FileWriter(save_dir, graph=sess.graph)
saver = tf.train.Saver(tf.global_variables())
for e in range(num_epochs): # for all epochs

    # LEARNING RATE SCHEDULING 
    sess.run(tf.assign(lr, learning_rate * (decay_rate ** e)))

    data_loader.reset_batch_pointer()
    state = sess.run(initial_state)
    for b in range(data_loader.num_batches):
        start = time.time()
        x, y = data_loader.next_batch()
        feed = {input_data: x, targets: y, initial_state: state}
        # Train!
        train_loss, state, _ = sess.run([cost, final_state, optm], feed)
        end = time.time()
        # PRINT 
        if b % 100 == 0:
            print ("%d/%d (epoch: %d), loss: %.3f, time/batch: %.3f"  
                   % (e * data_loader.num_batches + b
                    , num_epochs * data_loader.num_batches
                    , e, train_loss, end - start))
        # SAVE MODEL
        if (e * data_loader.num_batches + b) % save_every == 0:
            checkpoint_path = os.path.join(save_dir, 'model.ckpt')
            saver.save(sess, checkpoint_path
                       , global_step = e * data_loader.num_batches + b)
            print("model saved to {}".format(checkpoint_path))

0/99500 (epoch: 0), loss: 4.319, time/batch: 5.531
model saved to data/nine_dreams/model.ckpt
100/99500 (epoch: 0), loss: 2.949, time/batch: 3.866


In [ ]:
# IT TAKE A LOOOOOOOOT OF TIME